In [1]:
import requests
from io import BytesIO
import zipfile

import xmltodict
import json
DART_API_KEY = "f62b5aa9f2c561813b07d6e54b228f1eb5a7abe0"

## get all codes
# 참고: corp_code는 Dart에서 종목별로 부여한 고유번호로, 아래처럼 호출하면 모두 볼 수 있음(ref: http://blog.quantylab.com/2020-10-09-dart.html)

def get_corp_codes():
    api_url = 'https://opendart.fss.or.kr/api/corpCode.xml'
    res = requests.get(api_url, params={'crtfc_key': DART_API_KEY})
    z = zipfile.ZipFile(BytesIO(res.content))
    data_xml = z.read('CORPCODE.xml').decode('utf-8')
    data_odict = xmltodict.parse(data_xml)
    data_dict = json.loads(json.dumps(data_odict))
    data = data_dict.get('result', {}).get('list')
    return data

corp_codes = get_corp_codes()



In [8]:
## 만약 같은 이름의 회사 2개가 있을 경우 첫번째 값을 호출 (예를 들어 삼성물산은 2개가 있음)
def get_corp_code(corp_name):
    tmp = []
    for item in corp_codes:
        if item['corp_name'] == corp_name:
            tmp.append(item)
            # corp_code = item['corp_name']
            # print(item)
    return tmp[0]['corp_code']


## 증권신고서 주요사항 가져오기
def get_sec_summary(corp_name):
    url = 'https://opendart.fss.or.kr/api/estkRs.json'
    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': get_corp_code(corp_name),
        'bgn_de': '20160101',
        'end_de': '20221231'
    }

    result = requests.get(url, params=params).json()
    return result

{"status":"000","message":"정상","group":[{"title":"일반사항","list":[{"rcept_no":"20211224000319","corp_cls":"Y","corp_code":"01515323","corp_name":"LG에너지솔루션","sbd":"2022년 01월 18일 ~ 2022년 01월 19일","pymd":"2022년 01월 21일","sband":"2022년 01월 18일","asand":"2022년 01월 21일","asstd":"-","exstk":"-","exprc":"-","expd":"-","rpt_rcpn":"20211207000392"}]},{"title":"증권의종류","list":[{"rcept_no":"20211224000319","corp_cls":"Y","corp_code":"01515323","corp_name":"LG에너지솔루션","stksen":"기명식보통주","stkcnt":"42,500,000","fv":"500","slprc":"257,000","slta":"10,922,500,000,000","slmthn":"일반공모"}]},{"title":"인수인정보","list":[{"rcept_no":"20211224000319","corp_cls":"Y","corp_code":"01515323","corp_name":"LG에너지솔루션","stksen":"기명식보통주","actsen":"대표","actnmn":"케이비증권","udtcnt":"9,350,000","udtamt":"2,402,950,000,000","udtprc":"16,820,650,000","udtmth":"총액인수"},{"rcept_no":"20211224000319","corp_cls":"Y","corp_code":"01515323","corp_name":"LG에너지솔루션","stksen":"기명식보통주","actsen":"인수","actnmn":"하나금융투자","udtcnt":"425,000","udtamt":"10